In [12]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser

In [2]:
# 1. MARS NEWS SITE

# URL of page to be scraped -- we're going to SCRAP the WEB PAGE. Instead of using a path to a File.
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
#response
#print(response.text) #it will bring the text of the page we are scrapping

In [3]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

In [4]:
#Find ALL lists of all NEWS  --> NOT WORKING
#news = soup.find_all('li', class_="slide")
#print(news)

In [5]:
# first Title
news_title = soup.find("div",class_="content_title").text
print(news_title)

#¿SERÁ DEL SUGGESTED FEATURES?



Mars Helicopter Attached to NASA's Perseverance Rover




In [6]:
# first Parragraph text
news_p = soup.find("div", class_="rollover_description_inner").text 
print(news_p)

#¿SERÁ DEL SUGGESTED FEATURES?


The team also fueled the rover's sky crane to get ready for this summer's history-making launch.



In [8]:
items = soup.find_all("div", class_="rollover_description_inner") # ALL descriptions / DOESN'T SHOW ALL!!!
print(items)

[<div class="rollover_description_inner">
The team also fueled the rover's sky crane to get ready for this summer's history-making launch.
</div>, <div class="rollover_description_inner">
After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.
</div>, <div class="rollover_description_inner">
As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars.
</div>, <div class="rollover_description_inner">
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 
</div>, <div class="rollover_description_inner">
Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.
</div>, <div class="rollover_description_inner">
Jim Bridenstine addresse

In [23]:
# 2. JPL MARS SPACE IMAGE

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [24]:
#Click on button "FULL IMAGE"
browser.find_by_id('full_image').first.click()

In [29]:
#Click on button "MORE INFO"
browser.find_link_by_partial_text('more info').first.click()

In [46]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve full size image partial link (JPG format)
feature_image = soup.find("figure", class_="lede").a["href"]
print(feature_image)

/spaceimages/images/largesize/PIA16682_hires.jpg


In [47]:
#Link of full size image - with HTTPS:// link
featured_image_url ="https://www.jpl.nasa.gov/" + feature_image
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA16682_hires.jpg


In [48]:
# ANOTHER WAY TO GET THE IMAGE
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all elements that contain images links. The second one is in JPG format
images= soup.find_all('div', class_="download_tiff")[1]
print(images)

#Link of full size image - from PHOTOJOURNAL
image_url =images.a["href"]
print(image_url)

<div class="download_tiff">
<p>Full-Res JPG: <a href="//photojournal.jpl.nasa.gov/jpeg/PIA16682.jpg">PIA16682.jpg</a></p>
</div>
//photojournal.jpl.nasa.gov/jpeg/PIA16682.jpg
